In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score,precision_recall_fscore_support,precision_score,recall_score,f1_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras.backend as K
import tensorflow as tf
import os
import csv
import nltk
import spacy
import json

In [ ]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
SEQ_LEN=128
DIM_EMB=300

In [ ]:
corpora = '/content/drive/MyDrive/Colab Data/articles'
tagged_data=[]
irrelevant_docs=0
irrelevant_docs_address=[]
df_corpus={'Content':[],'Label':[]}

for subdir, dirs, files in os.walk(corpora):
  for file in files:
    path = os.path.join(subdir, file)
    
    with open(path,'r') as f:
      data=json.load(f)
      if data['candidateLabels']: #not an empty list thus relevant document

        if data['content']:
          df_corpus['Content'].append(data['content'].replace('\r', '').replace('\t', '').replace('\n','').replace('&#13;',''))
          df_corpus['Label'].append('relevant')

      else:#irrelevant document
        """df_corpus['Content'].append(data['content'])
        df_corpus['Label'].append('irrelevant')"""

        if data['content']:
          df_corpus['Content'].append(data['content'].replace('\r', '').replace('\t', '').replace('\n','').replace('&#13;',''))
          df_corpus['Label'].append('irrelevant')

In [ ]:
data=pd.DataFrame(df_corpus)

In [ ]:
data.head()

,Content,Label
0,Instead of creating unnecessary fear and paran...,irrelevant
1,The International Monetary Fund has given Bot...,irrelevant
2,10 October 2015 Last updated at: 5:05 AM M...,irrelevant
3,YAMAGUCHI – An outbreak of highly pathogenic ...,relevant
4,SEOUL South Korea has confirmed a fresh case o...,relevant


In [ ]:
X=data['Content'].to_numpy()
y=data['Label'].to_numpy()

In [ ]:
tokenizer = Tokenizer(num_words= 30000,lower=False)
label_encoder=LabelEncoder()

In [ ]:
tokenizer.fit_on_texts(data['Content'])

In [ ]:
label_encoder.fit(data['Label'])

LabelEncoder()

In [ ]:
label_encoder.inverse_transform([0])

array(['irrelevant'], dtype=object)

In [ ]:
X=data['Content']
y=label_encoder.transform(data['Label'])

In [ ]:
vectorizer = TfidfVectorizer(lowercase=False)

In [ ]:
X_vec=vectorizer.fit_transform(X)

In [ ]:
vocab=vectorizer.get_feature_names()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_vec.toarray()[0].shape

(18190,)

In [ ]:
embeddings_array=X_vec.toarray().T

In [ ]:
embeddings_array.shape

(18190, 503)

In [ ]:
pca=PCA(n_components=DIM_EMB)

In [ ]:
dense_embeddings=pca.fit_transform(embeddings_array)

In [ ]:
dense_embeddings.shape

(18190, 300)

In [ ]:
embeddings_dict={}

In [ ]:
for i in range(len(vocab)):
  embeddings_dict[vocab[i]]=dense_embeddings[i]

In [ ]:
embeddings_dict['flu'].shape

(300,)

In [ ]:
pd.DataFrame(embeddings_dict).head()

,00,000,000m2,000sq,000t,00144feab7de,00144feabdc0,009,00Pay,00Z,00ZLast,01,010,012,019,01T00,02,024,025,026,02908,02h30,03,03Z,04,041,04T11,04m,05,052,059,05T00,05T15,06,060km,062,069,06T00,06T13,06h00,...,zoning,zonă,zoo,zoonosis,zoonotic,zootehniei,Äripäev,Çelik,Élevage,Época,Éstas,În,Înmatriculare,Într,ámbitos,îmbolnăvire,în,înmatriculările,înregistrată,într,între,învecinate,înţepătura,úlceras,Łopaciuk,œCurrently,Şahin,Ştefăneşti,şi,ședintei,și,動植物防疫所,廖培志,張淑賢,畜衛所,萬丹,防檢局,陳保基,陳智雄,﨑県
0,0.060261,0.275573,-0.010310,-0.009168,0.054959,-0.012315,-0.010389,-0.008261,0.003675,0.031976,0.011745,0.007969,-0.011234,-0.005370,-0.011362,-0.006742,0.017702,-0.011149,-0.007328,-0.006368,-0.011297,-0.010634,-0.005242,-0.009585,0.006810,-0.008504,-0.010002,-0.009825,0.009745,-0.010002,-0.012267,-0.010082,-0.011530,0.010171,-0.009876,-0.010578,-0.013110,-0.009825,-0.009825,-0.010634,...,0.006403,-0.013491,0.006396,-0.008727,-0.001777,-0.013438,-0.010291,0.005197,-0.008909,-0.013467,-0.013449,-0.013573,-0.013477,-0.013477,-0.013468,-0.013461,-0.014152,-0.013477,-0.013461,-0.013461,-0.013461,-0.013461,-0.013461,-0.013449,-0.010359,-0.005700,-0.009705,-0.013491,-0.013865,-0.013477,-0.013828,-0.006523,-0.008381,-0.011086,-0.008381,-0.008381,-0.006523,-0.008381,-0.009131,-0.010133
1,-0.051860,0.078239,-0.000521,-0.002393,-0.093646,-0.000732,-0.003018,0.012301,-0.025097,-0.055789,-0.011573,0.005414,0.001858,0.003658,-0.000634,-0.007606,-0.029528,-0.000504,0.019149,0.005190,0.001547,-0.000222,-0.005022,-0.002228,-0.003464,0.008688,-0.005003,-0.003363,-0.011940,0.004279,0.000243,-0.001098,-0.000569,-0.009234,-0.002577,0.005272,0.000439,-0.003363,-0.003363,-0.000222,...,0.012995,0.000970,0.017496,0.003949,0.001981,0.001195,-0.000725,0.004426,0.001139,0.000808,0.000692,0.001374,0.000804,0.000804,0.000805,0.000773,0.004850,0.000804,0.000773,0.000773,0.000773,0.000773,0.000773,0.000692,-0.005902,-0.006098,0.001346,0.000970,0.003687,0.000804,0.002786,0.005582,0.005720,0.000691,0.005720,0.005720,0.005582,0.005720,0.004093,0.004590
2,0.047935,-0.003459,0.000486,0.003526,0.060072,-0.000130,-0.000905,0.008125,0.017024,0.043525,0.015446,0.015496,0.002246,-0.008106,-0.001696,0.007201,0.015407,-0.000890,0.017254,0.000520,0.000206,-0.001937,-0.003740,0.003965,0.028757,0.002916,0.002966,0.003302,0.021167,0.001630,-0.000866,0.003735,-0.001157,0.011434,0.002112,0.002512,0.000130,0.003302,0.003302,-0.001937,...,0.000654,0.000436,-0.002029,-0.000044,-0.008234,0.000643,0.001131,-0.011071,-0.002665,0.000468,0.000389,0.000804,0.000511,0.000511,0.000452,0.000373,0.002509,0.000511,0.000373,0.000373,0.000373,0.000373,0.000373,0.000389,0.008303,-0.009484,-0.001966,0.000436,0.001693,0.000511,0.001733,-0.002077,0.000673,-0.002748,0.000673,0.000673,-0.002077,0.000673,0.002508,0.001865
3,-0.029121,-0.038992,0.002113,-0.005779,0.020031,0.005203,0.015924,-0.004582,0.004695,-0.025026,-0.005677,0.001418,0.002752,-0.012742,0.000022,-0.006627,-0.014235,0.000210,0.010723,0.002771,0.001751,-0.000495,0.003436,-0.005277,0.007171,-0.007070,-0.000964,-0.003148,-0.024825,0.001452,-0.000773,-0.005763,0.000763,-0.015865,-0.003199,-0.001082,-0.000701,-0.003148,-0.003148,-0.000495,...,-0.035192,-0.001923,0.009456,0.000905,0.003413,-0.002359,-0.001559,-0.013491,-0.005920,-0.001379,-0.001123,-0.003103,-0.001530,-0.001530,-0.001423,-0.001394,-0.012662,-0.001530,-0.001394,-0.001394,-0.001394,-0.001394,-0.001394,-0.001123,-0.003240,0.002635,-0.003391,-0.001923,-0.008638,-0.001530,-0.007082,-0.001264,-0.000699,-0.001489,-0.000699,-0.000699,-0.001264,-0.000699,0.001619,0.001412
4,0.002923,0.054412,-0.000261,0.000395,0.008736,-0.005087,-0.015271,-0.003725,-0.018633,-0.002623,-0.000188,0.002791,0.000863,0.005885,-0.000780,-0.000951,-0.010049,-0.000532,-0.001500,0.000068,-0.000422,-0.001733,0.000878,-0.001388,0.001333,-0.004766,0.000399,0.000247,-0.001090,0.002051,-0.000591,0.000209,-0.000170,0.003868,-0.001606,0.001399,0.000381,0.000247,0.000247,-0.001733,...,0.016186,0.001004,0.016969,0.000005,-0.00711

In [ ]:
vocab[10000]

'displeasure'

In [ ]:
def document_vectorizer(document):
  embed_list=[]
  #print('LEN ',len(document[0]))
  for i in range(0,len(document[0])):
    word_index=document[0][i]
    word=tokenizer.sequences_to_texts([[word_index]])[0]
    #print(word,'\n')
    if word in embeddings_dict:
      #print(word,'\n')
      embed_list.append(embeddings_dict[word])
    else:
      #print('Haiko ',word,'\n')
      embed_list.append(np.zeros((DIM_EMB,)))
  return np.array(embed_list).flatten()

In [ ]:
embed=[]
for document in X:
  #print(document)
  sequences = tokenizer.texts_to_sequences([document])
  seq_document = pad_sequences(sequences, maxlen= SEQ_LEN, padding='post') 
  #print(seq_document)
  embed.append(document_vectorizer(seq_document))

In [ ]:
len(embed)

503

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(embed,y,test_size=0.2)

X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=0.2)

In [ ]:
cls_2=svm.SVC(kernel='rbf')

In [ ]:
cls_2.fit(X_train,y_train)

SVC()

In [ ]:
result = {}
for i in range(2,11):
  kfold = KFold(n_splits=i)
  c_validated = cross_val_score(cls_2,X_train,y_train,cv=kfold,scoring='f1') 
  result[i] = c_validated.mean()
print(result)

{2: 0.16761363636363635, 3: 0.23740510697032435, 4: 0.23778068396142488, 5: 0.2251672977457182, 6: 0.25531671989337895, 7: 0.2651920035748853, 8: 0.25693994902586237, 9: 0.2438123595244338, 10: 0.2220859695859696}


In [ ]:
list(result.values())

[0.16761363636363635,
 0.23740510697032435,
 0.23778068396142488,
 0.2251672977457182,
 0.25531671989337895,
 0.2651920035748853,
 0.25693994902586237,
 0.2438123595244338,
 0.2220859695859696]

In [ ]:
prediction=cls_2.predict(X_test)

In [ ]:
print(accuracy_score(y_test,prediction))

In [ ]:
precision_score(y_test,prediction,pos_label=1)

In [ ]:
recall_score(y_test,prediction,pos_label=1)

In [ ]:
f1_score(y_test,prediction,pos_label=1)